# Installing Missing modules

In [1]:
! pip install wfdb

     |████████████████████████████████| 122kB 5.9MB/s 


# Connecting to Drive

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries

In [35]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# Reading Data

In [62]:
df = pd.read_pickle('/content/drive/MyDrive/Database') # Full dataset : Cardiology_Dataset file 
#--> https://drive.google.com/file/d/1V9nNxMehtiwAjNVS-8usTP6Jol9jC6et/view?usp=sharing
df

,Label,Sequence
0,0,"[[-127, -162, -197, -229, -245, -254, -261, -2..."
1,0,"[[128, 157, 189, 226, 250, 257, 262, 265, 268,..."
2,0,"[[56, 73, 85, 93, 100, 107, 113, 117, 118, 117..."
3,2,"[[519, 619, 723, 827, 914, 956, 955, 934, 920,..."
4,2,"[[-188, -239, -274, -316, -356, -374, -380, -3..."
5,0,"[[-266, -316, -367, -407, -423, -423, -401, -3..."
6,0,"[[21, 22, 24, 26, 28, 31, 32, 34, 34, 35, 35, ..."
7,1,"[[-187, -236, -286, -340, -375, -396, -405, -4..."
8,2,"[[51, 56, 59, 63, 65, 62, 52, 38, 18, -10, -24..."
9,0,"[[-1028, -1225, -1418, -1599, -1747, -1808, -1..."


# EDA

In [48]:
df['Label'].value_counts()

0    14
1     6
2     4
3     1
Name: Label, dtype: int64

In [49]:
fig = px.pie(df, values='Label', names = 'Label', title='Repartition of ECG categories')
fig.show()

## Normal ECG

In [50]:
# To plot

# DataSet Splitting

In [57]:
# train, val = train_test_split(df, stratify=df.Label, test_size=0.2)
# print(f'Shape of training DataSet : {train.shape}')
# print(f'Shape of validation DataSet : {val.shape}')

In [58]:
# X_train = tf.ragged.constant(train.Sequence)
# y_train = tf.one_hot(train.Label,4)

# X_val = tf.ragged.constant(val.Sequence)
# y_val = tf.one_hot(val.Label,4)


In [59]:
# print(f'Maximal sequence length of training set : {X_train.bounding_shape()[-1]} ')
# print(f'Maximal sequence length of validation set : {X_val.bounding_shape()[-1]} ')

# Implementing Model

In [60]:
xx = tf.ragged.constant(df['Sequence'])

yy = tf.one_hot(df['Label'],4)

xx.bounding_shape()[-1]

<tf.Tensor: shape=(), dtype=int64, numpy=18000>

In [61]:
#max_seq = X_train.bounding_shape()[-1]
max_seq = xx.bounding_shape()[-1]
print(f'Maximum length sequence : {max_seq}')

model = keras.Sequential([
                          tf.keras.layers.Input(shape=[None, max_seq], dtype=tf.float32, ragged=True),
                          keras.layers.LSTM(units=64, activation='tanh', dropout=0.2, name = 'LSTM_1', return_sequences=True),
                          keras.layers.LSTM(units=256, activation='tanh', dropout=0.2, name = 'LSTM_2',return_sequences=True),
                          keras.layers.LSTM(units=100, activation='tanh', dropout=0.2, name = 'LSTM_3', return_sequences=False),
                          # tf.keras.layers.LSTM(4),
                          #tf.keras.layers.Flatten(name = 'Flatten'),
                          tf.keras.layers.Dense(units=4, activation='sigmoid', name = 'Dense_1')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.summary()

Maximum length sequence : 18000
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_1 (LSTM)                (None, None, 64)          4624640   
_________________________________________________________________
LSTM_2 (LSTM)                (None, None, 256)         328704    
_________________________________________________________________
LSTM_3 (LSTM)                (None, 100)               142800    
_________________________________________________________________
Dense_1 (Dense)              (None, 4)                 404       
Total params: 5,096,548
Trainable params: 5,096,548
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.fit(xx,yy, epochs=10)

Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 1.3867 - accuracy: 0.2800
Epoch 2/10
1/1 [==============================] - 0s 55ms/step - loss: 1.3848 - accuracy: 0.3600
Epoch 3/10
1/1 [==============================] - 0s 61ms/step - loss: 1.3762 - accuracy: 0.5200
Epoch 4/10
1/1 [==============================] - 0s 56ms/step - loss: 1.3662 - accuracy: 0.6000
Epoch 5/10
1/1 [==============================] - 0s 58ms/step - loss: 1.3643 - accuracy: 0.7200
Epoch 6/10
1/1 [==============================] - 0s 64ms/step - loss: 1.3612 - accuracy: 0.6000
Epoch 7/10
1/1 [==============================] - 0s 58ms/step - loss: 1.3538 - accuracy: 0.6000
Epoch 8/10
1/1 [==============================] - 0s 57ms/step - loss: 1.3385 - accuracy: 0.7200
Epoch 9/10
1/1 [==============================] - 0s 59ms/step - loss: 1.3363 - accuracy: 0.6800
Epoch 10/10
1/1 [==============================] - 0s 57ms/step - loss: 1.3178 - accuracy: 0.6400
